In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils import data
from tiny_imagenet import TinyImageNet

In [2]:
%reload_ext autoreload
%autoreload 2

## 步驟
- 讀取資料集
- 簡單 EDA
    - facets
- 定義目標 / loss function
- 定義模型
- 訓練模型
- 測試模型
- 視覺化 kernels / parameters

## 前處理資料
- 讀取資料
- 轉換（灰階處理、Augmentation、Crop）

注意在 validation 時我們不需要做 augmentation

In [3]:
root = './dataset'

In [11]:
# normalize 在現在有 batch-normalization 的情況下其實非必要
normalize = transforms.Normalize((.5, .5, .5), (.5, .5, .5))

augmentation = transforms.RandomApply([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(64)], p=.8)

training_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    augmentation,
    transforms.ToTensor(),
    normalize])

valid_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    transforms.ToTensor(),
    normalize])

In [12]:
training_set = TinyImageNet(root, 'train', transform=training_transform)
valid_set = TinyImageNet(root, 'val', transform=valid_transform)

In [13]:
training_set

Dataset TinyImageNet
    Number of datapoints: 100000
    Split: train
    Root Location: ./dataset
    Transforms (if any): Compose(
                             Lambda()
                             RandomApply(
                             p=0.8
                             RandomHorizontalFlip(p=0.5)
                             RandomRotation(degrees=(-10, 10), resample=False, expand=False)
                             RandomResizedCrop(size=(64, 64), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                         )
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None

## 訓練模型

In [7]:
params = {'batch_size': 64, 'shuffle': True, 'num_workers': 6}
# params = {'batch_size': 2, 'shuffle': True, 'num_workers': 1}
max_epochs = 100

In [8]:
training_loader = data.DataLoader(training_set, **params)

In [14]:
%%time
for epoch in range(1):
    for idx, (data, target) in enumerate(training_loader):
        print(f'{idx}/{len(training_loader)}', end='\r')

CPU times: user 1.06 s, sys: 718 ms, total: 1.78 s
Wall time: 1min 57s
